# Pregunta 1 — TWFE & Event-Study (R)

## Librerías

In [ ]:
library(tidyverse)
library(data.table)
library(fixest)
library(ggplot2)

theme_set(theme_minimal())

## Carga de Datos

In [ ]:
url <- "https://raw.githubusercontent.com/LOST-STATS/LOST-STATS.github.io/master/Model_Estimation/Data/Event_Study_DiD/bacon_example.csv"
df <- fread(url)

cat("Dataset Shape:", nrow(df), "x", ncol(df), "\n\n")
cat("Primeras filas:\n")
print(head(df, 10))
cat("\nStructura de datos:\n")
str(df)
cat("\nEstadísticas descriptivas:\n")
summary(df)

## a) TWFE - Regresión de Efectos Fijos Bidireccionales

**Especificación:**
$$Y_{it} = \alpha_i + \gamma_t + \beta \cdot D_{it} + X_{it}'\delta + \epsilon_{it}$$

Donde:
- $Y_{it}$ = asmrs (tasa de mortalidad ajustada por edad)
- $\alpha_i$ = efectos fijos por estado (stfips)
- $\gamma_t$ = efectos fijos por año (year)
- $D_{it}$ = indicador de tratamiento post (treated_post)
- $X_{it}$ = controles (pcinc, asmrh, cases)

### Exploración de la Estructura de Tratamiento

In [ ]:
cat("Años de adopción del tratamiento (_nfd):\n")
print(sort(unique(df$`_nfd`)))
cat("\nNúmero de unidades (estados):", df$stfips %>% n_distinct(), "\n")
cat("Rango temporal:", min(df$year), "-", max(df$year), "\n\n")
cat("Distribución por status de tratamiento:\n")
print(table(df$`_nfd`))

### Estimación TWFE

In [ ]:
df <- df %>%
  mutate(
    `_treated` = as.integer(`_nfd` > 0),
    post = as.integer(`_nfd` > 0 & year >= `_nfd`),
    treated_post = post * `_treated`
  )

twfe_model <- feols(
  asmrs ~ treated_post + pcinc + asmrh + cases | stfips + year,
  data = df,
  cluster = "stfips"
)

cat(paste(rep("=", 80), collapse = ""), "\n")
cat("REGRESIÓN TWFE - RESULTADOS\n")
cat(paste(rep("=", 80), collapse = ""), "\n")
print(twfe_model)

twfe_coef <- coef(twfe_model)["treated_post"]
twfe_se <- se(twfe_model)["treated_post"]

summary_twfe <- tibble(
  Variable = names(coef(twfe_model)),
  Coeficiente = coef(twfe_model),
  `Error Std.` = se(twfe_model),
  `t-stat` = coef(twfe_model) / se(twfe_model),
  `p-valor` = 2 * (1 - pnorm(abs(coef(twfe_model) / se(twfe_model)))),
  `IC 95% Inferior` = coef(twfe_model) - 1.96 * se(twfe_model),
  `IC 95% Superior` = coef(twfe_model) + 1.96 * se(twfe_model)
)

cat("\n", paste(rep("=", 80), collapse = ""), "\n")
cat("TABLA RESUMEN TWFE\n")
cat(paste(rep("=", 80), collapse = ""), "\n")
print(summary_twfe, n = Inf)
cat("\n*** Efecto del Tratamiento (treated_post):", sprintf("%.4f", twfe_coef), 
    "(SE:", sprintf("%.4f", twfe_se), ") ***\n")

## b) Preparación para Event-Study

### Paso 1: Crear la variable de tiempo relativo

In [ ]:
df_es <- df %>%
  mutate(
    treatment_year = `_nfd`,
    event_time = if_else(treatment_year > 0, year - treatment_year, NA_integer_)
  )

cat("Variable de tiempo relativo creada!\n\n")
cat("Muestra de datos con event_time:\n")
print(head(df_es %>% select(stfips, year, treatment_year, event_time, asmrs), 15))

### Paso 2: Tabla de frecuencias del tiempo relativo

In [ ]:
event_time_freq <- df_es %>%
  filter(!is.na(event_time)) %>%
  count(event_time, name = "Frecuencia") %>%
  arrange(event_time) %>%
  mutate(
    Porcentaje = round(100 * Frecuencia / sum(Frecuencia), 2),
    `Porcentaje Acum.` = cumsum(Porcentaje)
  )

cat(paste(rep("=", 70), collapse = ""), "\n")
cat("TABLA DE FRECUENCIAS - TIEMPO RELATIVO AL TRATAMIENTO\n")
cat(paste(rep("=", 70), collapse = ""), "\n")
print(event_time_freq, n = Inf)
cat("\nTotal de observaciones con event_time:", sum(event_time_freq$Frecuencia), "\n")
cat("Observaciones nunca tratadas:", sum(is.na(df_es$event_time)), "\n")
cat("Rango de tiempo relativo:", min(event_time_freq$event_time), "a", max(event_time_freq$event_time), "\n")

write.csv(event_time_freq, "../output/salida_1.csv", row.names = FALSE)
cat("\n✓ Tabla de frecuencias guardada en: ../output/salida_1.csv\n")

### Paso 3: Selección de límites para agrupar extremos

In [ ]:
valid_times <- df_es %>%
  filter(!is.na(event_time)) %>%
  pull(event_time)

cat(paste(rep("=", 70), collapse = ""), "\n")
cat("ANÁLISIS DE PERCENTILES - SELECCIÓN DE LÍMITES\n")
cat(paste(rep("=", 70), collapse = ""), "\n")

percentiles <- c(0.01, 0.05, 0.10, 0.25, 0.50, 0.75, 0.90, 0.95, 0.99)
perc_values <- quantile(valid_times, probs = percentiles, na.rm = TRUE)
for (i in seq_along(percentiles)) {
  cat(sprintf("Percentil %5.0f%%: %6.1f\n", percentiles[i] * 100, perc_values[i]))
}

cat("\nESTADÍSTICOS DESCRIPTIVOS:\n")
cat("Mínimo:  ", min(valid_times), "\n")
cat("Máximo:  ", max(valid_times), "\n")
cat("Media:   ", round(mean(valid_times), 2), "\n")
cat("Mediana: ", median(valid_times), "\n")
cat("D.E.:    ", round(sd(valid_times), 2), "\n")

LOWER_BOUND <- quantile(valid_times, probs = 0.10, na.rm = TRUE) %>% as.integer()
UPPER_BOUND <- quantile(valid_times, probs = 0.90, na.rm = TRUE) %>% as.integer()

cat("\n>>> Límites seleccionados (basados en percentiles 10%-90%):\n")
cat("LOWER_BOUND = ", LOWER_BOUND, "\n")
cat("UPPER_BOUND = ", UPPER_BOUND, "\n")
cat(paste(rep("=", 70), collapse = ""), "\n")

### Paso 4: Creación de variables dummy para el time relativo

In [ ]:
df_es <- df_es %>%
  mutate(
    event_time_binned = case_when(
      is.na(event_time) ~ NA_integer_,
      event_time < LOWER_BOUND ~ LOWER_BOUND,
      event_time > UPPER_BOUND ~ UPPER_BOUND,
      TRUE ~ event_time
    )
  )

dummy_cols <- c()
for (t in LOWER_BOUND:UPPER_BOUND) {
  col_name <- paste0("d_", t)
  df_es <- df_es %>%
    mutate(
      !!col_name := if_else(!is.na(event_time_binned) & event_time_binned == t, 1L, 0L)
    )
  dummy_cols <- c(dummy_cols, col_name)
}

cat(paste(rep("=", 70), collapse = ""), "\n")
cat("DUMMIES DE TIEMPO RELATIVO CREADAS\n")
cat(paste(rep("=", 70), collapse = ""), "\n")
cat("Total de dummies:", length(dummy_cols), "\n")
cat("Rango de tiempo relativo binned:", LOWER_BOUND, "a", UPPER_BOUND, "\n")
cat("\nDummies creadas:\n")
for (i in 1:length(dummy_cols)) {
  if (i %% 5 == 1) cat("\n")
  cat(sprintf("%-8s", dummy_cols[i]))
}
cat("\n")

dummy_sums <- df_es %>% select(all_of(dummy_cols)) %>% colSums()
cat("\nFrecuencias de dummies (primeros 10):\n")
for (i in 1:min(10, length(dummy_cols))) {
  cat(sprintf("%s: %4d  ", dummy_cols[i], dummy_sums[i]))
  if (i %% 5 == 0) cat("\n")
}
cat("\n")

# Excluir d_-1 como referencia
dummy_cols_for_reg <- dummy_cols[dummy_cols != "d_-1"]
cat("\nDummies para la regresión (excluyendo d_-1 como referencia):", length(dummy_cols_for_reg), "\n")
cat(paste(rep("=", 70), collapse = ""), "\n")

## c) Estimación del Event-Study

In [ ]:
formula_str <- paste(
  "asmrs ~",
  paste(dummy_cols_for_reg, collapse = " + "),
  "| stfips + year"
)

cat("Fórmula de regresión event-study:\n")
cat(formula_str, "\n\n")

es_model <- feols(
  as.formula(formula_str),
  data = df_es,
  cluster = "stfips"
)

cat("\n")
print(es_model)

es_coef <- tibble(
  term = names(es_model$coefficients),
  estimate = as.numeric(es_model$coefficients),
  se = as.numeric(es_model$se),
  t_stat = as.numeric(es_model$coefficients) / as.numeric(es_model$se),
  p_value = 2 * pt(abs(as.numeric(es_model$coefficients) / as.numeric(es_model$se)), 
                     df = nobs(es_model) - length(es_model$coefficients) - 1, lower.tail = FALSE),
  ci_lower = estimate - 1.96 * se,
  ci_upper = estimate + 1.96 * se
) %>%
  mutate(
    significant = case_when(
      p_value < 0.01 ~ "***",
      p_value < 0.05 ~ "**",
      p_value < 0.10 ~ "*",
      TRUE ~ ""
    )
  )

# Extraer tiempo relativo de los nombres de los coeficientes
es_results_df <- es_coef %>%
  mutate(
    event_time = as.integer(gsub("d_", "", term))
  ) %>%
  select(event_time, estimate, se, t_stat, p_value, ci_lower, ci_upper, significant) %>%
  arrange(event_time)

cat("\n")
cat(paste(rep("=", 85), collapse = ""), "\n")
cat("RESULTADOS DEL EVENT-STUDY - EFECTOS DINÁMICOS POR TIEMPO RELATIVO\n")
cat(paste(rep("=", 85), collapse = ""), "\n")
print(es_results_df, n = Inf)

write.csv(es_results_df, "../output/salida_2.csv", row.names = FALSE)
cat("\n✓ Resultados del event-study guardados en: ../output/salida_2.csv\n")

## d) Visualización del Event-Study

In [ ]:
plot_data <- es_results_df %>%
  mutate(
    pre_treatment = if_else(event_time < 0, 1, 0)
  )

p <- plot_data %>%
  ggplot(aes(x = event_time, y = estimate, color = factor(pre_treatment))) +
  # Zona de pre-tratamiento sombreada (gris claro)
  annotate(
    "rect",
    xmin = LOWER_BOUND - 0.5,
    xmax = -0.5,
    ymin = -Inf,
    ymax = Inf,
    fill = "lightgray",
    alpha = 0.2,
    color = NA
  ) +
  # Zona de post-tratamiento sombreada (muy claro)
  annotate(
    "rect",
    xmin = -0.5,
    xmax = UPPER_BOUND + 0.5,
    ymin = -Inf,
    ymax = Inf,
    fill = "lightyellow",
    alpha = 0.15,
    color = NA
  ) +
  # Línea en cero
  geom_hline(yintercept = 0, linetype = "dashed", color = "gray50", linewidth = 0.5) +
  # Línea en x = -0.5 (separación pre/post)
  geom_vline(xintercept = -0.5, linetype = "dashed", color = "darkred", linewidth = 0.4, alpha = 0.6) +
  # Barras de error
  geom_errorbar(
    aes(ymin = ci_lower, ymax = ci_upper),
    width = 0.2,
    position = position_dodge(0),
    linewidth = 0.6
  ) +
  # Puntos
  geom_point(size = 2.5, position = position_dodge(0)) +
  # Línea conectando puntos
  geom_line(aes(group = 1), linewidth = 0.5, alpha = 0.7) +
  # Etiquetas y temas
  scale_color_manual(
    values = c("0" = "#2E86AB", "1" = "#A23B72"),
    labels = c("0" = "Post-tratamiento", "1" = "Pre-tratamiento"),
    name = "Período"
  ) +
  labs(
    title = "Event-Study: Efectos dinámicos del tratamiento sobre ASMRS",
    subtitle = "Con intervalos de confianza al 95% (líneas), períodos pre/post sombreados",
    x = "Tiempo relativo al tratamiento (años)",
    y = "Efecto sobre ASMRS (coeficiente)",
    caption = "Período de referencia: t = -1. Barras de error: ±1.96 × SE con clustering en stfips."
  ) +
  theme_minimal() +
  theme(
    plot.title = element_text(size = 14, face = "bold", hjust = 0.5),
    plot.subtitle = element_text(size = 11, hjust = 0.5, color = "gray40"),
    axis.title = element_text(size = 11, face = "bold"),
    legend.position = "top",
    panel.grid.major.x = element_blank(),
    panel.grid.minor = element_blank()
  )

print(p)

ggsave("../output/salida_3.png", plot = p, width = 12, height = 6, dpi = 300)
cat("\n✓ Gráfico del event-study guardado en: ../output/salida_3.png\n")

## Resumen de resultados

### Salidas generadas:
- `salida_1.csv`: Tabla de frecuencias del tiempo relativo al tratamiento
- `salida_2.csv`: Coeficientes del event-study con intervalos de confianza al 95%
- `salida_3.png`: Gráfico del event-study con visualización de períodos pre/post-tratamiento

### Interpretación:
El event-study estima efectos dinámicos del tratamiento, permitiendo verificar:
1. **Pre-tendencias**: Los coeficientes pre-tratamiento (t < -1) deben ser cercanos a cero e insignificantes
2. **Efecto promedio**: El coeficiente t = 0 (año de adopción) representa el impacto en el primer año
3. **Evolución temporal**: Los coeficientes posteriores (t > 0) muestran cómo evoluciona el efecto en el tiempo

### Especificación de la regresión:
$$ASMRS_{it} = \beta_0 + \sum_{k=\text{LOWER}}^{\text{UPPER}} \beta_k D^k_{it} + \gamma_i + \delta_t + \epsilon_{it}$$

donde:
- $D^k_{it}$ = indicadores de tiempo relativo (excluida la categoría k = -1)
- $\gamma_i$ = efectos fijos de entidad (stfips)
- $\delta_t$ = efectos fijos de tiempo (year)
- $\epsilon_{it}$ = término de error con clustering a nivel de entidad